In [2]:
#Отобразите все записи из таблицы company по компаниям, которые закрылись.
Select *
FROM company
WHERE status = 'closed' 

#Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total.
SELECT funding_total
FROM company 
WHERE country_code = 'USA' AND category_code = 'news'
ORDER BY funding_total DESC 

#Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash' AND EXTRACT(YEAR FROM acquired_at :: DATE) BETWEEN 2011 AND 2013 

#Отобразите имя, фамилию и названия аккаунтов людей в поле network_username, у которых названия аккаунтов начинаются на 'Silver'
SELECT first_name,last_name,twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%'

#Выведите на экран всю информацию о людях, у которых названия аккаунтов в поле network_username содержат подстроку 'money', а фамилия начинается на 'K'.
SELECT *
FROM people 
WHERE twitter_username LIKE '%money%' AND last_name LIKE 'K%'

#Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.
SELECT country_code ,SUM(funding_total) 
FROM company 
GROUP BY country_code
ORDER BY SUM(funding_total) DESC 

#Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
#Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.
SELECT funded_at,MIN(raised_amount) AS MI,MAX(raised_amount) AS MA
FROM funding_round 
GROUP BY funded_at
HAVING MIN(raised_amount) != 0 AND MIN(raised_amount) != MAX(raised_amount)

#Создайте поле с категориями:
#Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
#Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
#Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
#Отобразите все поля таблицы fund и новое поле с категориями.
SELECT *,
       CASE
       WHEN invested_companies >= 100 THEN 'high_activity'
       WHEN invested_companies >= 20 and invested_companies < 100 THEN 'middle_activity'
       WHEN  invested_companies < 20 THEN 'low_activity'
       END 
FROM fund
#Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.
SELECT 
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
           END AS activity,
           ROUND(AVG(investment_rounds))
FROM fund
GROUP BY activity
ORDER BY ROUND(AVG(investment_rounds));

#Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
#Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 
#Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.
SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
FROM (SELECT *
     FROM fund
     WHERE EXTRACT(YEAR FROM founded_at::DATE) BETWEEN 2010 AND 2012) AS f 
GROUP BY country_code
HAVING MIN(invested_companies) > 0 
ORDER BY AVG(invested_companies)DESC
LIMIT 10 

#Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.
SELECT  p.first_name,p.last_name,instituition
FROM people AS p 
LEFT JOIN education AS e ON   p.id = e.person_id

#Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.
SELECT c.name,
       COUNT(DISTINCT(instituition))  AS CDI
FROM company AS c 
INNER  JOIN people AS p ON c.id = p.company_id   
INNER JOIN  education AS e ON p.id  = e.person_id 
GROUP BY c.name
ORDER BY CDI DESC 
LIMIT 5 

#Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.
SELECT DISTINCT(name) 
FROM company AS c
INNER JOIN funding_round AS f ON c.id = f.company_id 
WHERE c.status = 'closed' AND f.is_first_round = 1 AND  f.is_last_round = 1

#Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.
SELECT DISTINCT(p.id)
FROM company AS c 
INNER JOIN funding_round AS f ON f.company_id =c.id 
INNER JOIN people AS p  ON c.id =p.company_id
WHERE c.status = 'closed' AND f.is_first_round = 1 AND f.is_last_round = 1

#Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.
SELECT DISTINCT(p.id),instituition
FROM company AS c 
INNER JOIN funding_round AS f ON f.company_id =c.id 
INNER JOIN people AS p ON c.id =p.company_id
INNER JOIN education AS e ON p.id =e.person_id
WHERE c.status = 'closed' AND f.is_first_round = 1 AND f.is_last_round = 1

#Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.
SELECT p.id,
       COUNT(instituition)
FROM people AS p 
LEFT JOIN education AS e ON p.id =e.person_id
WHERE p.company_id IN (SELECT c.id
                      FROM company AS c
                      INNER JOIN funding_round AS f ON c.id =f.company_id 
                      WHERE c.status = 'closed' AND is_first_round = 1 AND 
                      is_last_round = 1
                      GROUP BY c.id)
GROUP BY p.id
HAVING COUNT(DISTINCT e.instituition) >0;

#Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.
WITH base AS
(SELECT p.id,
COUNT(e.instituition)
FROM people AS p 
LEFT JOIN education AS e ON p.id =e.person_id
WHERE p.company_id IN (SELECT c.id
                      FROM company AS c
                      JOIN funding_round AS f ON c.id =f.company_id 
                      WHERE c.status = 'closed' AND is_first_round = 1 AND 
                      is_last_round = 1
                      GROUP BY c.id)
                      GROUP BY p.id
                      HAVING COUNT(DISTINCT e.instituition) >0 )
SELECT AVG(COUNT)
FROM base 

#Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Socialnet.
WITH base AS
(SELECT p.id,
COUNT(e.instituition)
FROM people AS p 
RIGHT JOIN education AS e ON p.id =e.person_id
WHERE p.company_id IN (SELECT id
                      FROM company
                      WHERE name = 'Facebook')
                      GROUP BY p.id)
SELECT AVG(COUNT)
FROM base 

#Составьте таблицу из полей:
#name_of_fund — название фонда;
#name_of_company — название компании;
#amount — сумма инвестиций, которую привлекла компания в раунде.
#В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.
WITH 
C_1 AS (SELECT f.name AS name_of_fund,
        c.name AS name_of_company,
        fr.raised_amount AS amount
        FROM fund AS f 
        INNER JOIN investment AS i ON f.id = i.fund_id 
        INNER JOIN funding_round AS FR ON i.funding_round_id = fr.id
        INNER JOIN company AS c ON fr.company_id = c.id
        WHERE c.milestones > 6 AND  EXTRACT(YEAR FROM fr.funded_at::DATE ) BETWEEN 2012 AND 2013)

SELECT *
FROM C_1

#Выгрузите таблицу, в которой будут такие поля:
#название компании-покупателя;
#сумма сделки;
#название компании, которую купили;
#сумма инвестиций, вложенных в купленную компанию;
#доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
#Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
#Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

SELECT c_1.name AS name_1,
       a.price_amount as price_1,
       c_2.name AS name_2,
       c_2.funding_total AS price_2,
       ROUND(a.price_amount/sum (c_2.funding_total)) AS D
FROM acquisition AS a 
LEFT JOIN company AS c_1 ON  a.acquiring_company_id = c_1 .id 
LEFT JOIN company AS c_2 ON  a.acquired_company_id = c_2.id 
WHERE c_2.funding_total>0
GROUP BY  name_1,price_1, name_2,price_2,c_2.funding_total
ORDER BY price_1 DESC
LIMIT 10;
#Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.
SELECT c.name,EXTRACT(MONTH FROM fr.funded_at :: DATE) 
FROM company AS c
INNER JOIN funding_round AS fr ON c.id = fr.company_id
WHERE c.category_code = 'social' AND EXTRACT(YEAR FROM fr.funded_at :: DATE) BETWEEN 2010 AND 2013 AND fr.raised_amount != 0 

#Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
#номер месяца, в котором проходили раунды;
#количество уникальных названий фондов из США, которые инвестировали в этом месяце;
#количество компаний, купленных за этот месяц;
#общая сумма сделок по покупкам в этом месяце.
WITH fun AS 
(SELECT EXTRACT(MONTH FROM fr.funded_at:: DATE) AS month,
        COUNT(DISTINCT f.id) AS code 
FROM fund AS f
LEFT JOIN investment AS i ON f.id = i.fund_id
LEFT JOIN funding_round AS fr ON i.funding_round_id = fr.id
WHERE f.country_code = 'USA' AND EXTRACT (YEAR FROM fr.funded_at:: DATE)
BETWEEN 2010 AND 2013 
GROUP BY month),
        
acq AS 
(SELECT EXTRACT(MONTH FROM a.acquired_at:: DATE) AS month,
        COUNT(a.acquired_company_id) AS count_acquired,
        SUM(a.price_amount) as price
FROM acquisition AS a
FULL OUTER JOIN company AS c ON a.acquired_company_id = c.id
WHERE  EXTRACT (YEAR FROM a.acquired_at:: DATE)
BETWEEN 2010 AND 2013 
GROUP BY month)
        
SELECT FUN.month,FUN.code,ACQ.count_acquired,ACQ.price
FROM fun AS FUN 
LEFT JOIN acq AS ACQ ON FUN.month = ACQ.month 

#Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.
WITH
inv_2011 AS (SELECT AVG(funding_total) AS AVG_2011,
                    country_code 
             FROM company
             WHERE EXTRACT (YEAR FROM founded_at::DATE) = 2011 
             GROUP BY country_code),  -- сформируйте первую временную таблицу
inv_2012 AS (SELECT AVG(funding_total) AS AVG_2012,
                    country_code 
             FROM company
             WHERE EXTRACT (YEAR FROM founded_at::DATE) = 2012
             GROUP BY country_code),
inv_2013 AS (SELECT AVG(funding_total) AS AVG_2013,
                    country_code 
             FROM company
             WHERE EXTRACT (YEAR FROM founded_at::DATE) = 2013
             GROUP BY country_code)         
SELECT inv_2011.country_code,
       inv_2011.AVG_2011,
       --inv_2012.country_code,
       inv_2012.AVG_2012,
       --inv_2013.country_code,
       inv_2013.AVG_2013-- отобразите нужные поля
FROM inv_2011 -- укажите таблицу
INNER JOIN inv_2012 on inv_2011.country_code= inv_2012.country_code
INNER JOIN inv_2013 on inv_2012.country_code = inv_2013.country_code
ORDER BY inv_2011.AVG_2011 DESC


SyntaxError: invalid syntax (3161167638.py, line 2)